In [1]:
def run_tkreg(dms, img3, cr_sigclip=0, run2=True):
    # Set tweakreg parameters
    img3.tweakreg.tolerance = 0.5
    img3.tweakreg.searchrad = 1
    img3.tweakreg.separation = 0.01
    
    # Detect sources for each DM
    srcs = []
    for dm_idx, dm in enumerate(dms):
        # Here, we still use the original image to avoid border effect
        # we only select high-S/N sources for accurate astrometry
        src, _, _, _, _ = det_src(dm, sigma_threshold=2, 
                                  clean_edge=True, cr_sigclip=cr_sigclip)
        srcs.append(src)
        # Transform xy to radec
        ras, decs = dms[dm_idx].meta.wcs.get_transform('detector', 'world')(src['x'], src['y'])
        # Attach to the datamodel 
        dms[dm_idx].catalog = Table([src['x'], src['y'], ras, decs], 
                                    names=['x', 'y', 'RA', 'DEC'])
    # Run the code
    tkreg_res = img3.tweakreg.run(dms.copy())
    
    # A 2nd run?
    if run2:
        for dm_idx, dm in enumerate(tkreg_res):
            # Get the sources from 1st run
            src = srcs[dm_idx]
            # Transform xy to radec
            ras, decs = tkreg_res[dm_idx].meta.wcs.get_transform('detector', 'world')(src['x'], src['y'])
            # Attach to the datamodel 
            tkreg_res[dm_idx].catalog = Table([src['x'], src['y'], ras, decs], 
                                        names=['x', 'y', 'RA', 'DEC'])
        tkreg_res = img3.tweakreg.run(tkreg_res)
        
    return tkreg_res